# Aufgabe 10 - Monte Carlo Methods mit Frozen Lake
15.01.2022, Thomas Iten


**Content**
0. Setup
1. Create FrozenLakeHelper
2. Explore Blackjack Environment
3. Play random policy and testing policy
4. Limit Stochastic Methode
5. Monte Carlo Prediction
6. Monte Carlo Control
7. Monte Carlo Testing

**References**
- https://www.deep-teaching.org/notebooks/reinforcement-learning/exercise-monte-carlo-frozenlake-gym
- https://adityajain.me/blogs/monte-carlo-and-temporal-difference.html
- https://www.analyticsvidhya.com/blog/2018/11/reinforcement-learning-introduction-monte-carlo-learning-openai-gym/

## 0. Setup

### Imports

In [1]:
import gym
import numpy as np
from collections import defaultdict


## 1. Create FrozenLakeHelper class


In [2]:
class FrozenLakeHelper():
    """Some helper methods used throughout this notebook."""

    def render(self, env, display_mode="brackets", print_result=True, legend=False):
        """
        IntelliJ notebooks to not render the color of the current position correct.
        Details see: https://youtrack.jetbrains.com/issue/PY-32191

        Therfore we use this customized render methode with two simple display modes.

        :param env: The current environment to render it's fields.
        :param display_mode: display current position with "brackets" or in "lowercase"
        :param print_result: print the last action and result
        :param legend: print the legend
        :return: lastaction as text and fields with marked current position
        """

        # init data
        row, col = env.s // env.ncol, env.s % env.ncol
        desc = env.desc.tolist()
        desc = [[c.decode("utf-8") for c in line] for line in desc]

        actions = ["Left", "Down", "Right", "Up"]
        action = "Init" if env.lastaction is None else actions[env.lastaction]

        # format display mode
        indicator = None
        if display_mode == "brackets":
            desc[row][col] = "[{}]".format(desc[row][col])
            desc = [[ (" {} ".format(c) if len(c) == 1 else c) for c in line ] for line in desc]
            indicator = "[]"
        elif display_mode == "lowercase":
            desc[row][col] = (desc[row][col]).lower()
            indicator = "lowercase"

        # print result
        if print_result:
            if legend:
                print("Last action:", action)
            else:
                print(action + ":")
            for line in desc:
                for pos in line:
                    print(pos, end="")
                print("")
            if legend:
                print("Legend: S=Start, F=Frozen (safe), H=Hole, G=Goal, " + indicator + "=Current Position")
            print("")

        # return result
        return action, desc


    def print_field_positions(self):
        print("Field positons:")
        print("[ 0] [ 1] [ 2] [ 3]")
        print("[ 4] [ 5] [ 6] [ 7]")
        print("[ 8] [09] [10] [11]")
        print("[12] [13] [14] [15]")

    def print_actions(self):
        print("Actions:")
        print("[0] Left")
        print("[1] Down")
        print("[2] Right")
        print("[3] Up")

    def print_Q(self, Q):
        print("Field: Left        Down       Right      Up")
        for field in Q:
            print(f"{field : >5}", end="")
            print(":", Q[field])

# Create helper instance
helper = FrozenLakeHelper()


## 2. Explore Frozen Lake Environment

### Frozen Lake environment

In [3]:
env = gym.make('FrozenLake-v1', is_slippery=False)

print("Action space:")
print(env.action_space)
print("")

helper.print_actions()
print("")

print("Observation space:")
print(env.observation_space)
print("")

helper.print_field_positions()


Action space:
Discrete(4)

Actions:
[0] Left
[1] Down
[2] Right
[3] Up

Observation space:
Discrete(16)

Field positons:
[ 0] [ 1] [ 2] [ 3]
[ 4] [ 5] [ 6] [ 7]
[ 8] [09] [10] [11]
[12] [13] [14] [15]


### Reset and initial state

In [4]:
env.reset()    # reset the environment the set agent to start state
helper.render(env, legend=True)
print()

Last action: Init
[S] F  F  F 
 F  H  F  H 
 F  F  F  H 
 H  F  F  G 
Legend: S=Start, F=Frozen (safe), H=Hole, G=Goal, []=Current Position




## 3. Play random policy and testing policy

### Play random policy

The step returns:
- new_state: new state after action (random_action) taken in current state
- reward: reward obtained after taken action (random_action) in current state and entering new state (new_state)
- done: bool flag, true if goal or hole is reached
- info: slippery probability, baseline is 1/3

In [5]:
env.reset()
helper.render(env)

# Play till done or maximum 6 iterations reached
for i in range(1,7):
    print("Step:", i)
    random_action = env.action_space.sample() #samples a random action
    new_state, reward, done, info = env.step(random_action) #agent takes action (random_action)
    helper.render(env)
    if done:
        break

Init:
[S] F  F  F 
 F  H  F  H 
 F  F  F  H 
 H  F  F  G 

Step: 1
Down:
 S  F  F  F 
[F] H  F  H 
 F  F  F  H 
 H  F  F  G 

Step: 2
Left:
 S  F  F  F 
[F] H  F  H 
 F  F  F  H 
 H  F  F  G 

Step: 3
Right:
 S  F  F  F 
 F [H] F  H 
 F  F  F  H 
 H  F  F  G 



### Testing policy

The problem is said to be solved, a good policy found, if over a sufficient number of episodes (>100)
a mean reward of >0.7 is reached.

The code below tests this for a given policy.

In [6]:
def test_performance(policy, nb_episodes=100):
    sum_returns = 0
    for i in range(nb_episodes):
        state  = env.reset()
        done = False
        while not done:
            action = policy(state)
            state, reward, done, info = env.step(action)
            if done:
                sum_returns += reward
    return sum_returns/nb_episodes


Lets set and test a random policy. Since later we will work with aϵ−policy we always will start by defining
a policy as a dictionary (state-action pairs) or array (index-value pairs) and than wrap the dictionary
or array into a function.

> We see that the number is far lower than 0.7 and therefor the policy is not good/ optimal.

In [7]:
# Define random policy
random_policy = lambda s: env.action_space.sample()

# test random policy
mean_reward = test_performance(random_policy)
print("Test random policy:")
print("Mean reward =", mean_reward)


Test random policy:
Mean reward = 0.01


## 4. Limit Stochastic Methode

In this section, you will write your own implementation of MC prediction (for estimating the action-value function).

### Define generate_episode (with random policy)
Generates the starting policy.

In [8]:
def generate_episode(env):
    episode = []
    state = env.reset()
    while True:
        action = env.action_space.sample()
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

### Play generate_episode (with random policy)
Play the game n episodes and print results of each episode.

In [9]:
print("Play 3 times and print episodes with: (state, action, reward)")

for i in range(3):
    episode = generate_episode(env)
    print(episode)

Play 3 times and print episodes with: (state, action, reward)
[(0, 3, 0.0), (0, 0, 0.0), (0, 3, 0.0), (0, 0, 0.0), (0, 0, 0.0), (0, 1, 0.0), (4, 1, 0.0), (8, 2, 0.0), (9, 2, 0.0), (10, 3, 0.0), (6, 2, 0.0)]
[(0, 2, 0.0), (1, 3, 0.0), (1, 1, 0.0)]
[(0, 1, 0.0), (4, 1, 0.0), (8, 2, 0.0), (9, 2, 0.0), (10, 3, 0.0), (6, 0, 0.0)]


## 5. Monte Carlo Prediction

### Define mc_prediction

**Input Arguments:**
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `generate_episode`: This is a function that returns an episode of interaction.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1 inclusive (default value: `0.8`).
- `print_n_episode`: Each n episode will be printed to system out (default value: `1`).

**Return:**
- `Q`: This is a dictionary (of one-dimensional arrays)
- where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.

In [10]:
def mc_prediction_q(env, num_episodes, generate_episode, gamma=0.8, print_n_episode=1):

    # initialize empty dictionaries of arrays
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    # loop over episodes
    for i_episode in range(1, num_episodes+1):

        # monitor progress
        is_print = i_episode == 1 or i_episode % print_n_episode == 0
        if is_print:
            print("# ================================================================")
            print("# Episode {}/{}".format(i_episode, num_episodes))
            print("# ================================================================")

        # generate an episode
        episode = generate_episode(env)

        # obtain the states, actions, and rewards
        states, actions, rewards = zip(*episode)

        # prepare for discounting
        discounts = np.array([gamma**i for i in range(len(rewards))])
        if is_print:
            print("Prepare discounting:")
            print("discounts:", discounts)
            print("rewards:", rewards)
            print("states:", states)
            print("")

        # update the sum of the returns, number of visits, and action-value
        # function estimates for each state-action pair in the episode
        for i, (state, action) in enumerate(zip(states, actions)):
            n_steps_after_state = len(rewards[i:])
            if is_print:
                print("rewards[i:]:", rewards[i:])
                print("discounts[:n_steps_after_state]:", discounts[:n_steps_after_state])
            returns_sum[state][action] += sum(rewards[i:]*discounts[:n_steps_after_state])
            N[state][action] += 1.0
            Q[state][action] = returns_sum[state][action] / N[state][action]
            if is_print:
                print("Q-Value:", Q[state][action])
                print("State:", state)
                print("Action:", action)
                print("")
    return Q

### Test mc_prediction

In [11]:
num_episodes = 3
Q = mc_prediction_q(env, num_episodes, generate_episode)

# ================================================================
# Episode 1/3
# ================================================================
Prepare discounting:
discounts: [1.       0.8      0.64     0.512    0.4096   0.32768  0.262144]
rewards: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
states: (0, 1, 0, 1, 2, 2, 3)

rewards[i:]: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
discounts[:n_steps_after_state]: [1.       0.8      0.64     0.512    0.4096   0.32768  0.262144]
Q-Value: 0.0
State: 0
Action: 2

rewards[i:]: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
discounts[:n_steps_after_state]: [1.      0.8     0.64    0.512   0.4096  0.32768]
Q-Value: 0.0
State: 1
Action: 0

rewards[i:]: (0.0, 0.0, 0.0, 0.0, 0.0)
discounts[:n_steps_after_state]: [1.     0.8    0.64   0.512  0.4096]
Q-Value: 0.0
State: 0
Action: 2

rewards[i:]: (0.0, 0.0, 0.0, 0.0)
discounts[:n_steps_after_state]: [1.    0.8   0.64  0.512]
Q-Value: 0.0
State: 1
Action: 2

rewards[i:]: (0.0, 0.0, 0.0)
discounts[:n_steps_after_state]: [1.   0.8

### Play mc_prediction

In [12]:
num_episodes = 100000
print_n_episodes = 20000

# obtain the action-value function
Q = mc_prediction_q(env, num_episodes, generate_episode, print_n_episode=print_n_episodes)

# ================================================================
# Episode 1/100000
# ================================================================
Prepare discounting:
discounts: [1.         0.8        0.64       0.512      0.4096     0.32768
 0.262144   0.2097152  0.16777216 0.13421773 0.10737418 0.08589935
 0.06871948 0.05497558]
rewards: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
states: (0, 0, 1, 2, 2, 2, 2, 3, 3, 2, 3, 3, 2, 3)

rewards[i:]: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
discounts[:n_steps_after_state]: [1.         0.8        0.64       0.512      0.4096     0.32768
 0.262144   0.2097152  0.16777216 0.13421773 0.10737418 0.08589935
 0.06871948 0.05497558]
Q-Value: 0.0
State: 0
Action: 3

rewards[i:]: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
discounts[:n_steps_after_state]: [1.         0.8        0.64       0.512      0.4096     0.32768
 0.262144   0.2097152  0.16777216 0.13421773 0.

### Show Q-Value
Show Q with state and probabilities per action

In [13]:
helper.print_field_positions()
print()

helper.print_Q(Q)
print()


Field positons:
[ 0] [ 1] [ 2] [ 3]
[ 4] [ 5] [ 6] [ 7]
[ 8] [09] [10] [11]
[12] [13] [14] [15]

Field: Left        Down       Right      Up
    0: [0.00122896 0.00248166 0.0013838  0.00135066]
    1: [0.0013686  0.         0.00431315 0.00128558]
    2: [0.00161723 0.01477777 0.00150881 0.00460725]
    3: [0.0047191  0.         0.00178379 0.00163326]
    4: [0.00257205 0.00875482 0.         0.00136826]
    8: [0.00863522 0.         0.03130008 0.00279844]
    9: [0.00691463 0.08152864 0.067979   0.        ]
    6: [0.         0.069667   0.         0.00524931]
   13: [0.         0.07726199 0.28829045 0.03046507]
   10: [0.0321065  0.29689617 0.         0.01576785]
   14: [0.07612059 0.29630067 1.         0.06635169]



## 6. Monte Carlo Control

In this section, you will write your own implementation of constant-$\alpha$ MC control.  

Your algorithm has four arguments:
- `env`: This is an instance of an OpenAI Gym environment.
- `num_episodes`: This is the number of episodes that are generated through agent-environment interaction.
- `alpha`: This is the step-size parameter for the update step.
- `gamma`: This is the discount rate.  It must be a value between 0 and 1, inclusive (default value: `1`).

The algorithm returns as output:
- `Q`: This is a dictionary (of one-dimensional arrays) where `Q[s][a]` is the estimated action value corresponding to state `s` and action `a`.
- `policy`: This is a dictionary where `policy[s]` returns the action that the agent chooses after observing state `s`.

**Eval vs. Improvement**

Das kann nicht strikt getrennt werden.
Fausregel:
- Alles was Tabelle erstellt ist Evaluierung
- Alles was Q-Vaule anpasst ist Improvment


### generate_episode_from_Q, get_probs, update_Q

In [14]:
def generate_episode_from_Q(env, Q, epsilon, nA):
    """ generates an episode from following the epsilon-greedy policy """
    episode = []
    state = env.reset()

    while True:
        action = np.random.choice(np.arange(nA), p=get_probs(Q[state], epsilon, nA)) \
                                    if state in Q else env.action_space.sample()
        next_state, reward, done, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode


def get_probs(Q_s, epsilon, nA): # epsilon greedy
    """ obtains the action probabilities corresponding to epsilon-greedy policy """
    policy_s = np.ones(nA) * epsilon / nA # wenn epsilon 1 ist zu Beginn und ich habe 5 Aktionen, dann ist meine Policy [1/5, 1/5,..., 1/5]
    # print("policy_s:", policy_s)
    best_a = np.argmax(Q_s)
    policy_s[best_a] = 1 - epsilon + (epsilon / nA)
    # print("episode :", episode)
    # print("policy_s:", policy_s)
    # print("best_a  :", best_a)

    return policy_s

def update_Q(env, episode, Q, alpha, gamma, is_print): # policy improvement oder control
    """ updates the action-value function estimate using the most recent episode """
    states, actions, rewards = zip(*episode)
    # prepare for discounting
    discounts = np.array([gamma**i for i in range(len(rewards)+1)])
    for i, state in enumerate(states):
        n_steps_after_state = len(rewards[i:])
        old_Q = Q[state][actions[i]] 
        # Q[state][actions[i]] = old_Q + alpha*(sum(rewards[i:]*discounts[:-(1+i)]) - old_Q)
        # - Alpha mindert den Discount, wir passen die Prediction
        #   + oder - je nachem ob wir zuwenig oder zuviel predicted haben
        # - Q old wurde mit der gleichen Formel berechnet.
        # - Man könnte auch den Mittelwert nehmen, aber wir wollen das "geschickter" machen, fein granularer!
        Q[state][actions[i]] = old_Q + alpha*(sum(rewards[i:]*discounts[:n_steps_after_state]) - old_Q)
        if is_print:
            print('state:', state)
            print('action:', actions[i])
            print('q-value:', Q[state][actions[i]])
    return Q

### mc_prediction_control

In [15]:
def mc_prediction_control(env, num_episodes, alpha, gamma=0.05,
                          eps_start=1.0, eps_decay=.99999, eps_min=0.05,
                          print_n_episode=print_n_episodes):
    nA = env.action_space.n
    # initialize empty dictionary of arrays
    Q = defaultdict(lambda: np.zeros(nA))
    epsilon = eps_start
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        is_print = i_episode == 1 or i_episode % print_n_episode == 0
        if is_print:
            print("# ================================================================")
            print("# Episode {}/{}".format(i_episode, num_episodes))
            print("# ================================================================")
        # set the value of epsilon
        epsilon = max(epsilon*eps_decay, eps_min)
        # generate an episode by following epsilon-greedy policy

        # Start
        episode = generate_episode_from_Q(env, Q, epsilon, nA)

        # update the action-value function estimate using the episode
        Q = update_Q(env, episode, Q, alpha, gamma, is_print) # prediction und eigentllich control, weil Q table die Vorlage für die nächste Action Selection ist

    # determine the policy corresponding to the final action-value function estimate
    policy = dict((k,np.argmax(v)) for k, v in Q.items()) # control - Q table wird gelesen
    return policy, Q

### obtain the estimated optimal policy and action-value function

Use the cell below to obtain the estimated optimal policy and action-value function.
Note that you should fill in your own values for the `num_episodes` and `alpha` parameters.

In [16]:
num_episodes = 10000
print_n_episodes = 5000

alpha = 0.002
policy, Q = mc_prediction_control(env, num_episodes, alpha, print_n_episode=print_n_episodes)


# ================================================================
# Episode 1/10000
# ================================================================
state: 0
action: 3
q-value: 0.0
state: 0
action: 0
q-value: 0.0
state: 0
action: 0
q-value: 0.0
state: 0
action: 3
q-value: 0.0
state: 0
action: 0
q-value: 0.0
state: 0
action: 3
q-value: 0.0
state: 0
action: 2
q-value: 0.0
state: 1
action: 2
q-value: 0.0
state: 2
action: 2
q-value: 0.0
state: 3
action: 3
q-value: 0.0
state: 3
action: 0
q-value: 0.0
state: 2
action: 2
q-value: 0.0
state: 3
action: 2
q-value: 0.0
state: 3
action: 1
q-value: 0.0
# ================================================================
# Episode 5000/10000
# ================================================================
state: 0
action: 3
q-value: 2.031318954299623e-11
state: 0
action: 2
q-value: 8.490540837654538e-10
state: 1
action: 0
q-value: 2.483103007509795e-11
state: 0
action: 3
q-value: 2.0272563163910237e-11
state: 0
action: 0
q-value: 3.81351580028218

### show Q

In [17]:
helper.print_field_positions()
print()

helper.print_Q(Q)
print()

Field positons:
[ 0] [ 1] [ 2] [ 3]
[ 4] [ 5] [ 6] [ 7]
[ 8] [09] [10] [11]
[12] [13] [14] [15]

Field: Left        Down       Right      Up
    0: [3.93738363e-11 1.53393007e-09 8.25093142e-10 2.61134399e-11]
    1: [2.15769227e-11 0.00000000e+00 7.37887751e-08 2.31182956e-09]
    2: [2.22493288e-10 3.15122315e-06 9.40876212e-11 3.55717052e-08]
    3: [2.03049219e-08 0.00000000e+00 8.75413371e-10 3.57506431e-11]
    4: [1.78331390e-09 1.20161389e-07 0.00000000e+00 4.55599007e-11]
    8: [9.99729865e-08 0.00000000e+00 6.85002423e-06 2.86819002e-09]
    9: [4.29772814e-08 1.41128326e-04 1.98636195e-04 0.00000000e+00]
    6: [0.00000000e+00 1.52455528e-04 0.00000000e+00 3.17749363e-08]
   10: [3.01209764e-06 9.81198583e-03 0.00000000e+00 2.31093437e-06]
   14: [7.39389255e-05 5.11627953e-03 4.38181688e-01 1.07904445e-04]
   13: [0.00000000e+00 1.10383530e-04 6.77356519e-03 2.68596809e-06]



### show policy

In [18]:
helper.print_actions()
print()

policy


Actions:
[0] Left
[1] Down
[2] Right
[3] Up



{0: 1, 1: 2, 2: 1, 3: 0, 4: 1, 8: 2, 9: 2, 6: 1, 10: 1, 14: 2, 13: 2}

## 7. Monte Carlo Testing

In [19]:
mc_policy = lambda s: policy[s]

# Tests
print("Test random policy: Mean reward =", test_performance(random_policy))
print("Test mc policy    : Mean reward =", test_performance(mc_policy))


Test random policy: Mean reward = 0.0
Test mc policy    : Mean reward = 1.0


---
__The end.__